In [94]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
from pathlib import Path
import warnings

from AOSCMcoupling.convergence_checker import vector_norm, relative_error, relative_criterion

In [95]:
class OASISPreprocessor:
    """Preprocessor for OASIS Output Files from EC-Earth SCM runs.

    Drops the horizontal dimension which is added for NEMO but meaningless.
    If `origin` is provided: converts `time` coordinate to valid datetime objects,
    computed relative to the simulation start date, with optional time shift
    """

    def __init__(
        self, origin: pd.Timestamp = None, time_shift: pd.Timedelta = pd.Timedelta(0)
    ):
        """Constructor.

        :param origin: start date (+ time) of the simulation
        :type origin: pd.Timestamp
        :param time_shift: time shift to apply for local time, default: 0
        :type time_shift: pd.Timedelta, optional
        """
        self.origin = origin
        self.time_shift = time_shift

    def preprocess(self, ds: xr.Dataset) -> xr.DataArray:
        """Preprocess function for use with `xr.open_mfdataset()`.

        :param ds: dataset as loaded from disk
        :type ds: xr.Dataset
        :return: preprocessed DataArray (only one variable!)
        :rtype: xr.DataArray
        """
        ds = ds.isel(ny=0, nx=0)

        source_file = Path(ds.encoding["source"])
        iteration = int(source_file.parent.name.split("_")[-1])
        ds = ds.expand_dims(
            iteration=[iteration],
        )

        if self.origin is not None:
            time_data = np.array(ds.time.data, dtype="timedelta64[s]")
            ds = ds.assign_coords(time=self.origin + time_data + self.time_shift)
        return ds


In [96]:
def set_style():
    font_dir = Path("/Users/valentina/dev/aoscm/tex-gyre-heros")
    for path in font_dir.glob("*.otf"):
        font_manager.fontManager.addfont(path)
    plt.style.use("aoscm-experiments/stylesheet.mpl")
set_style()

In [97]:
preprocessor = OASISPreprocessor(pd.Timestamp("2014-07-01"), pd.Timedelta(-7, "h"))

In [98]:
coupling_vars = [
    "A_TauX_oce",
    "A_TauY_oce",
    "A_TauX_ice",
    "A_TauY_ice",
    "A_Qs_mix",
    "A_Qns_mix",
    "A_Qs_ice",
    "A_Qns_ice",
    "A_Precip_liquid",
    "A_Precip_solid",
    "A_Evap_total",
    "A_Evap_ice",
    "A_dQns_dT",
    "O_SSTSST",
    "O_TepIce",
    "O_AlbIce",
    "OIceFrc",
    "OIceTck",
    "OSnwTck",
]

In [99]:
def check_null(da: xr.DataArray) -> bool:
    if da.isnull().all():
        return True
    if not da.any():
        return True
    return False

In [101]:
output_dir = Path("output_cosmos")
exp_id = "TOPS"
netcdf_files = [file for file in output_dir.glob(f"{exp_id}_*/*.nc")]
oasis_files = [file for file in netcdf_files if any(cv in file.name for cv in coupling_vars)]

ds = xr.open_mfdataset(oasis_files, preprocess=preprocessor.preprocess)

relative_errors_2 = relative_error(ds, ds.isel(iteration=-1), ds.isel(iteration=-1), ord=2)
relative_errors_inf = relative_error(ds, ds.isel(iteration=-1), ds.isel(iteration=-1), ord=np.inf)

for var_name, v_relative_error_2 in relative_errors_2.items():
    v_relative_error_inf = relative_errors_inf[var_name]
    if check_null(v_relative_error_2) or check_null(v_relative_error_inf):
        print(f"{var_name}: All values NaN or 0, no plot created!")
        continue
    fig, ax = plt.subplots()
    v_relative_error_2.plot(ax=ax, ls="none", marker=".", label="2-norm")
    v_relative_error_inf.plot(ax=ax, ls="none", marker=".", label="max-norm")
    ax.set(
        yscale="log",
        ylabel="Relative Error",
        title=f"{exp_id}: SWR Convergence for {var_name}",
        xlabel="Iteration",
    )
    ax.legend()
    fig.savefig(f"plots/{exp_id}_{var_name}_convergence.png", dpi=250)
    plt.close()

A_Precip_liquid: All values NaN or 0, no plot created!
